This file retrieves and stores MAL data - top 50 shows for now

In [1]:
## load libraries

import numpy as np
import pandas as pd

from bs4 import BeautifulSoup
import requests
import time, os

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

chromedriver = "C:\\Users\\vi_ci\\Downloads\\chromedriver\\chromedriver.exe" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

In [2]:
def hover(self):
    wd = webdriver.connection
    element = wd.find_element_by_link_text(self.locator)
    hov = ActionChains(wd).move_to_element(element)
    hov.perform()

In [3]:
def MAL_loadtopanime(num_pages):
    '''
    Load the top X anime from MAL based on number of pages, 50 per page
    So X = (num_pages * 50)
    Returns a soup of the list of top X anime
    '''
    topanime_text = ""
    for counter in range(1, num_pages + 1):
        limit = (counter - 1) * 50
        url = "https://myanimelist.net/topanime.php?limit=" + str(limit)
        response = requests.get(url)
        if response.status_code != 200:
            print('Error reading page ', counter , ' of MAL Top ', num_pages * 50)
        else:
            topanime_text += response.text
    return BeautifulSoup(topanime_text, 'html5').find(class_="top-ranking-table").find_all(class_="detail")

In [4]:
soup_mal_top50 = MAL_loadtopanime(1)

Testing to retrieve data from the first item (defining functions in the meantime)

In [5]:
def MAL_initEntry_top(soup):
    '''
    Takes the soup of an entry in the MAL top 50
    Returns a dictionay entry with Title and URL of the anime
    '''
    entry = {}
    entry['Title'] = soup.find(class_="hoverinfo_trigger").text
    entry['URL'] = soup.find(class_="hoverinfo_trigger").get('href')
    return entry

In [6]:
number1 = MAL_initEntry_top(soup_mal_top50)

print(number1['Title'])
print(number1['URL'])

Fullmetal Alchemist: Brotherhood
https://myanimelist.net/anime/5114/Fullmetal_Alchemist__Brotherhood


Go to number1\['URL'\] and pull the soup

In [7]:
def MAL_retrieveEntry(entry):
    '''
    Takes a dictionary that has URL and Title of the MAL anime
    Returns a soup of the anine page in question
    '''
    response = requests.get(entry['URL'])
    if response.status_code != 200:
        print('Encountered ' + str(response.status_code) + ' error reading ' + entry['Title'])
        return -1
    else:
        return BeautifulSoup(response.text, 'html5')

In [8]:
soup_mal_number1 = MAL_retrieveEntry(number1)

Retrieve more items from the first item

In [9]:
number1sidebar = soup_mal_number1.find(id='content')

The below gives a list of all fields in the left column of the anime show page

In [10]:
number1sidebar.find_all(class_="dark_text")

[<span class="dark_text">English:</span>,
 <span class="dark_text">Synonyms:</span>,
 <span class="dark_text">Japanese:</span>,
 <span class="dark_text">Type:</span>,
 <span class="dark_text">Episodes:</span>,
 <span class="dark_text">Status:</span>,
 <span class="dark_text">Aired:</span>,
 <span class="dark_text">Premiered:</span>,
 <span class="dark_text">Broadcast:</span>,
 <span class="dark_text">Producers:</span>,
 <span class="dark_text">Licensors:</span>,
 <span class="dark_text">Studios:</span>,
 <span class="dark_text">Source:</span>,
 <span class="dark_text">Genres:</span>,
 <span class="dark_text">Duration:</span>,
 <span class="dark_text">Rating:</span>,
 <span class="dark_text">Score:</span>,
 <span class="dark_text">Ranked:</span>,
 <span class="dark_text">Popularity:</span>,
 <span class="dark_text">Members:</span>,
 <span class="dark_text">Favorites:</span>]

Cycle through the headers in the left column to get all entries. This does not take in multiple entries for a single column, only the first entry

In [81]:
def MAL_retrieveSidebar(anime_dict, soup):
    '''
    Returns anime_dict with additional raw data for the sidebar of the anime entry 
    '''
    sidebar = soup.find(id='content')
    headers = sidebar.find_all(class_="dark_text")
    for header in headers:
        column_name = header.text.strip()[:-1]
        # print(column_name) # error-checking
        entry = header.next_sibling.strip()
        # print(entry) # error-checking
        if(entry.strip() == "None found,"): # no entries
            entry = []
        elif(entry == ""):
            # special case for score
            if(column_name == 'Score'):
                entry = header.findNext().text
            else:
                entry_soup = header.findNext('a')
                # print(entry_soup.text, " ||| ", entry_soup.findNext('a'), " ||| ", entry_soup.findNext().name) # error-checking
                
                # create a list of items if more than one entry; signifed by the plural in column_name
                if(column_name[-1] != 's'):
                    entry = entry_soup.text
                else:
                    entry = [entry_soup.text]
                    while(entry_soup.findNext().name != 'div'):
                        entry_soup = entry_soup.findNext('a')
                        entry.append(entry_soup.text)
        anime_dict[column_name] = entry
        # print('---')
    return anime_dict

In [12]:
number1 = MAL_retrieveSidebar(number1, soup_mal_number1)

In [13]:
number1

{'Title': 'Fullmetal Alchemist: Brotherhood',
 'URL': 'https://myanimelist.net/anime/5114/Fullmetal_Alchemist__Brotherhood',
 'English': 'Fullmetal Alchemist: Brotherhood',
 'Synonyms': 'Hagane no Renkinjutsushi: Fullmetal Alchemist, Fullmetal Alchemist (2009), FMA, FMAB',
 'Japanese': '鋼の錬金術師 FULLMETAL ALCHEMIST',
 'Type': 'TV',
 'Episodes': '64',
 'Status': 'Finished Airing',
 'Aired': 'Apr 5, 2009 to Jul 4, 2010',
 'Premiered': 'Spring 2009',
 'Broadcast': 'Sundays at 17:00 (JST)',
 'Producers': ['Aniplex',
  'Square Enix',
  'Mainichi Broadcasting System',
  'Studio Moriken'],
 'Licensors': ['Funimation', 'Aniplex of America'],
 'Studios': ['Bones'],
 'Source': 'Manga',
 'Genres': ['Action',
  'Military',
  'Adventure',
  'Comedy',
  'Drama',
  'Magic',
  'Fantasy',
  'Shounen'],
 'Duration': '24 min. per ep.',
 'Rating': 'R - 17+ (violence & profanity)',
 'Score': '9.23',
 'Ranked': '#1',
 'Popularity': '#4',
 'Members': '1,737,987',
 'Favorites': '148,834'}

Aired, Duration, Members and Favorites needs post-processing

In [14]:
def MAL_ppDuration(duration_entry):
    '''
    Takes in the raw duration entry in the form (xx min.) (xx hr.) (per ep)
    Returns total number of minutes per episode
    '''
    duration_array = duration_entry.split()
    duration_norm = 0
    if 'min.' in duration_array:
        duration_norm += int(duration_array[duration_array.index('min.') - 1])
    if 'hr.' in duration_array:
        duration_norm += 60 * int(duration_array[duration_array.index('hr.') - 1])
    return duration_norm

In [15]:
def MAL_ppAired(anime_dict):
    '''
    Takes in the original anime sidebar dictionary
    Return the dictionary with 'Started' and 'Ended' columns added in Timestamp format
    '''
    def MAL_toDatetime(date_string):
        if '?' not in date_string:
            try:
                return pd.to_datetime(aired_array[0], format="%b %d, %Y")
            except:
                try:
                    return pd.to_datetime(aired_array[0], format="%b, %Y")
                except:
                    return pd.to_datetime(aired_array[0], format="%Y")
        else:
            return np.nan

    aired_array = anime_dict['Aired'].split(' to ')
    anime_dict['Started'] = MAL_toDatetime(aired_array[0])
    if len(aired_array) > 1:
        anime_dict['Ended'] = MAL_toDatetime(aired_array[1])
    return anime_dict

In [16]:
def remove_commas(input_str):
    return int(input_str.strip().replace(',',''))

def MAL_ppSidebar(sidebar_dict):
    '''
    Postprocessing of MAL sidebar
    Take in the unprocessed sidebar dictionary
    Return the processed sidebar dictionary
    '''

    if(sidebar_dict['Episodes'] == 'Unknown'):
        sidebar_dict['Episodes'] = np.nan
    else:
        sidebar_dict['Episodes'] = int(sidebar_dict['Episodes'])
    sidebar_dict['Duration'] = MAL_ppDuration(sidebar_dict['Duration'])
    sidebar_dict = MAL_ppAired(sidebar_dict)
    sidebar_dict['Members'] = remove_commas(sidebar_dict['Members'])
    sidebar_dict['Favorites'] = remove_commas(sidebar_dict['Favorites'])

    return sidebar_dict

In [17]:
number1 = MAL_ppSidebar(number1)

Now retrieve the details from the topbar of the anime show page

In [18]:
def MAL_retrieveTopbar(anime_dict, soup):
    '''
    Retrieves the score and voters from the topbar of the anime entry
    Return the modified anime_dict
    '''

    topbar = soup.find(class_='anime-detail-header-stats')
    
    if(anime_dict['Score'] == 'N/A'):
        anime_dict['Score'] = np.nan 
        anime_dict['Voters'] = np.nan # no score means no one voted
    else:
        anime_dict['Score'] = float(topbar.find(class_='score-label').text)
        anime_dict['Voters'] = remove_commas(topbar.find(class_='score').get('data-user').split()[0])

    return anime_dict

In [19]:
number1 = MAL_retrieveTopbar(number1, soup_mal_number1)
print(number1['Score'])
print(number1['Voters'])

9.23
1040597


To add related anime:

In [20]:
def MAL_retrieveRelated(anime_dict, soup):
    '''
    Retrieves related anime from the anime entry
    Returns the modified anime_dict
    '''
    related = soup.find(class_='anime_detail_related_anime')
    if(related is None): # nothing to add, so return
        return anime_dict
    related_rows = related.find_all('tr')
    for item in related_rows:
        header = item.find('td').text.strip()[:-1]
        entry = item.find('td').find_next().text.strip()
        anime_dict[header] = [item.strip() for item in entry.split(',')]
    
    return anime_dict

In [21]:
number1 = MAL_retrieveRelated(number1, soup_mal_number1)

Test to see that all data is captured

In [22]:
number1

{'Title': 'Fullmetal Alchemist: Brotherhood',
 'URL': 'https://myanimelist.net/anime/5114/Fullmetal_Alchemist__Brotherhood',
 'English': 'Fullmetal Alchemist: Brotherhood',
 'Synonyms': 'Hagane no Renkinjutsushi: Fullmetal Alchemist, Fullmetal Alchemist (2009), FMA, FMAB',
 'Japanese': '鋼の錬金術師 FULLMETAL ALCHEMIST',
 'Type': 'TV',
 'Episodes': 64,
 'Status': 'Finished Airing',
 'Aired': 'Apr 5, 2009 to Jul 4, 2010',
 'Premiered': 'Spring 2009',
 'Broadcast': 'Sundays at 17:00 (JST)',
 'Producers': ['Aniplex',
  'Square Enix',
  'Mainichi Broadcasting System',
  'Studio Moriken'],
 'Licensors': ['Funimation', 'Aniplex of America'],
 'Studios': ['Bones'],
 'Source': 'Manga',
 'Genres': ['Action',
  'Military',
  'Adventure',
  'Comedy',
  'Drama',
  'Magic',
  'Fantasy',
  'Shounen'],
 'Duration': 24,
 'Rating': 'R - 17+ (violence & profanity)',
 'Score': 9.23,
 'Ranked': '#1',
 'Popularity': '#4',
 'Members': 1737987,
 'Favorites': 148834,
 'Started': Timestamp('2009-04-05 00:00:00'),
 '

Now read the top 50 into a dictionary list

In [35]:
def MAL_createdict_top(soup):
    '''
    Takes in the soup of top anime and returns a dictionary list 
    '''
    mal_top = []
    for anime_soup in soup:

        # proceses the entry in the top 50 page(s) and gets the related page
        mal_entry = MAL_initEntry_top(anime_soup) 
        soup_mal_entry = MAL_retrieveEntry(mal_entry)
        
        # processes the anime page
        mal_entry = MAL_retrieveSidebar(mal_entry, soup_mal_entry)
        mal_entry = MAL_ppSidebar(mal_entry)
        mal_entry = MAL_retrieveTopbar(mal_entry, soup_mal_entry)
        mal_entry = MAL_retrieveRelated(mal_entry, soup_mal_entry)

        mal_top.append(mal_entry)
    
    return mal_top

In [ ]:
mal_top50 = MAL_createdict_top(soup_mal_top50)

Put the required columns into a dataframe

In [36]:
mal_df = pd.DataFrame(mal_top50)

In [80]:
mal_df.head(5)

,Title,URL,English,Japanese,Type,Episodes,Status,Aired,Producers,Licensors,...,Genres,Duration,Rating,Score,Ranked,Popularity,Members,Favorites,Started,Voters
0,A Brightening Life,https://myanimelist.net/anime/40628/A_Brighten...,A brightening life,A brightening life,Movie,1,Finished Airing,2010,"None found,","None found,",...,"[Drama, Music]",8,PG - Children,NaN,#10255,#15794,102,1,2010-01-01,NaN


Save it!

In [38]:
mal_df.to_pickle('mal_top50_mvp.pkl')

Now let's pull data from the alphabetical list

In [47]:
def MAL_loadanime_char(num_pages, char):
    '''
    Load the first X anime starting with char from MAL based on number of pages, 50 per page
    So X = (num_pages * 50)
    To load anime with non-alphabetical starts, use '.'
    Returns a soup of the list of the first X anime starting with char
    '''
    animechar_text = ""
    for counter in range(1, num_pages + 1):
        limit = (counter - 1) * 50
        url = "https://myanimelist.net/anime.php?letter=" + char + "&show=" + str(limit)
        response = requests.get(url)
        if response.status_code != 200:
            print('Error reading page ', counter , ' of MAL first ', num_pages * 50, ' starting with ', char)
        else:
            animechar_text += response.text
    return BeautifulSoup(animechar_text, 'html5').find_all(class_="picSurround")

In [48]:
def MAL_initEntry_alpha(soup):
    '''
    Takes the soup of an entry in the MAL alphabetical list
    Returns a dictionay entry with Title and URL of the anime
    '''
    entry = {}
    entry['Title'] = soup.find("img").get('alt')
    entry['URL'] = soup.get('href')
    return entry

In [ ]:
soup_mal_first50A_list = MAL_loadanime_char(1, 'A')

In [86]:
def MAL_createdict_firstchar(soup):
    '''
    Takes in the soup of first X anime starting with char and returns a dictionary list 
    '''
    mal_firstchar = []
    for anime_soup in soup:

        anime_soup = anime_soup.find(class_="hoverinfo_trigger")

        # proceses the entry in the top 50 page and gets the related page
        mal_entry = MAL_initEntry_alpha(anime_soup) 
        soup_mal_entry = MAL_retrieveEntry(mal_entry)
        
        # processes the anime page
        mal_entry = MAL_retrieveSidebar(mal_entry, soup_mal_entry)
        mal_entry = MAL_ppSidebar(mal_entry)
        mal_entry = MAL_retrieveTopbar(mal_entry, soup_mal_entry)
        mal_entry = MAL_retrieveRelated(mal_entry, soup_mal_entry)

        mal_firstchar.append(mal_entry)
    return mal_firstchar

In [ ]:
mal_first50A = MAL_createdict_firstchar(soup_mal_first50A_list)

In [87]:
mal_df = pd.DataFrame(mal_first50A)

In [88]:
mal_df.iloc[0]

Title                                                 A Brightening Life
URL                    https://myanimelist.net/anime/40628/A_Brighten...
English                                               A brightening life
Japanese                                              A brightening life
Type                                                               Movie
Episodes                                                               1
Status                                                   Finished Airing
Aired                                                               2010
Producers                                                             []
Licensors                                                             []
Studios                                                               []
Source                                                          Original
Genres                                                    [Drama, Music]
Duration                                           

In [89]:
mal_df.head(5)

,Title,URL,English,Japanese,Type,Episodes,Status,Aired,Producers,Licensors,...,Alternative version,Synonyms,Premiered,Broadcast,Adaptation,Other,Prequel,Parent story,Side story,Spin-off
0,A Brightening Life,https://myanimelist.net/anime/40628/A_Brighten...,A brightening life,A brightening life,Movie,1.0,Finished Airing,2010,[],[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,A Christmas Song,https://myanimelist.net/anime/39086/A_Christma...,A Christmas Song,A Christmas Song,Music,1.0,Finished Airing,"Nov 29, 2012",[Avex Entertainment],[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A Kite,https://myanimelist.net/anime/320/A_Kite,Kite,A KITE（カイト）,OVA,2.0,Finished Airing,"Feb 25, 1998 to Oct 25, 1998","[Green Bunny, BEAM Entertainment]",[Media Blasters],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,A Log Day of Timbre,https://myanimelist.net/anime/38712/A_Log_Day_...,A Log Day of Timbre,A Log Day of Timbre,ONA,1.0,Finished Airing,"Feb 25, 2011",[],[],...,[Timbre A to Z],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,A New Journey,https://myanimelist.net/anime/39057/A_New_Journey,NaN,A New Journey,ONA,1.0,Finished Airing,"Jan 18, 2019",[],[],...,NaN,Season 2019: A New Journey | League of Legends,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [90]:
mal_df.to_pickle('mal_first50a_mvp.pkl')